In [16]:
#Adjustments AI 
#Sample data creator


In [2]:
#Shared common config
SpecialFields=["AdjustmentType","AdjustmentName","AdjustedField"]
AuditFields=["OldVal","NewVal"]
SampleFields=["CustomerID","CustomerType","RiskCategory","IsDefaulted","CountryCode"]
NumericFields=["Drawn","Undrawn"]
RatioFields=["PD"]
AdjustmentTypes=["Amend"]
AdjustmentNames=["GRC_EndMonth","Fix_Broken_Batch","Credit_Uplift","Defaulting_Reclassify","Default"]
AdjustedFields=SampleFields
CustomerID=["CUST238947","CUST563201","CUST981372","CUST450896","CUST712035","CUST103289","CUST665428","CUST359147","CUST804231","CUST192654"]
CustomerType=["Retail","Corporate","SME"]
RiskCategory=["Low","Medium","High"]
IsDefaulted=["0","1"]
CountryCode=["US","GB","DE"]
FieldMapping={"CustomerID":CustomerID,"CustomerType":CustomerType,"RiskCategory":RiskCategory,"IsDefaulted":IsDefaulted,"CountryCode":CountryCode}

#Define adjustment rules
AdjustmentRules=[]
AdjustmentRules.append([[{"CustomerType":"Corporate"},{"RiskCategory":"High"}],[0.75,"Market Downturn"]])


In [3]:
import pandas as pd
import numpy as np

#Sample Data Generator
records=1000

#Hide warmings from pandas
pd.options.mode.chained_assignment = None  # default='warn'

def GetRandomFieldValue(field):
    return np.random.choice(FieldMapping[field],1)[0]

#Checks if this row matches any adjustment rule, if so returns the [{fieldToUpdate:NewValue},AdjustmentComment]
def ValidateGetAdjustmentRule(row):
    for rule in AdjustmentRules:
        criteria=rule[0]
        adjustment=rule[1]

        #Check all criteria match
        allMatch=True
        for crit in criteria:
            #Split crit dictionary into key and value
            critField=list(crit.keys())[0]
            critValue=crit[critField]

            #Check if row matched
            if row[critField]!=critValue:
                allMatch=False

    return allMatch,adjustment


df=pd.DataFrame(columns=SampleFields+SpecialFields+AuditFields)
for mapping in FieldMapping:
    df[mapping]=np.random.choice(FieldMapping[mapping],records)
df["AdjustmentType"]=np.random.choice(AdjustmentTypes,records)
df["AdjustmentName"]=np.random.choice(AdjustmentNames,records)

#Adjusted field------------------------------------------------------------------------
df["AdjustedField"]=np.random.choice(AdjustedFields,records)
#df["AdjustedField"]="PD"

for field in NumericFields:
    #Insert new column after first 1
    df[field]=np.random.randint(1000000, size=records)
for field in RatioFields:
    df[field]=np.random.randint(100, size=records)/100

#Add in audit fields
df["OldVal"]=np.nan
df["NewVal"]=np.nan

#Loop over each row and generate a new value for Oldval based on field
for index, row in df.iterrows():
    adjField=row["AdjustedField"]

    #Check if this row matches any adjustment rule
    matchesRule,rule = ValidateGetAdjustmentRule(row)

    #APPLY ADJUTSMENTS--------------------------------------
    if adjField not in NumericFields and adjField not in RatioFields:

        if matchesRule:
            
            #Split rule dictionary into key and value
            ruleTarget=rule[0]
            ruleComment=rule[1]
            
            #df["AdjustmentType"][index]="DERP"#
            # #df["OldVal"][index]=row[ruleField]
            df["AdjustmentName"][index]=ruleComment
            df["NewVal"][index]=ruleTarget
        else:
            #Random "noise" adjustments
            #df["OldVal"][index]=row[adjField]
            df["NewVal"][index]=GetRandomFieldValue(adjField)

# #Reshuffle the numeric fields to the start
df=df.reindex(columns=SpecialFields+NumericFields+RatioFields+SampleFields+AuditFields)

#Remove OldVal Field
df=df.drop(columns=["OldVal"])

# #Write to CSV file
df.to_csv("SampleData.csv",index=False)
df


,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,Fix_Broken_Batch,CustomerID,727741,167749,0.15,CUST712035,Retail,High,0,US,CUST981372
1,Amend,Default,RiskCategory,322590,409969,0.15,CUST450896,SME,Low,0,DE,Medium
2,Amend,Defaulting_Reclassify,RiskCategory,916339,631088,0.95,CUST712035,Retail,Medium,1,GB,High
3,Amend,Defaulting_Reclassify,IsDefaulted,474573,963829,0.50,CUST192654,Corporate,Medium,1,DE,0
4,Amend,Market Downturn,CustomerType,402089,742127,0.12,CUST192654,Corporate,High,1,DE,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Amend,Default,CustomerID,499655,799747,0.78,CUST712035,Corporate,Medium,1,DE,CUST665428
996,Amend,Default,IsDefaulted,843487,320775,0.74,CUST238947,SME,High,0,US,0
997,Amend,GRC_EndMonth,CountryCode,68954,745314,0.00,CUST192654,SME,Low,0,US,US
998,Amend,Defaulting_Reclassify,CustomerID,249039,355545,0.90,CUST359147,Corporate,Medium,0,DE,CUST563201


3